In [1]:
%pylab notebook
import datacube
from datetime import datetime, timedelta
import pandas as pd
from fc.fractional_cover import fractional_cover
import pyproj
import csv

dc = datacube.Datacube(app='nbart-fieldsites')

Populating the interactive namespace from numpy and matplotlib


In [2]:
fieldsites = pd.read_csv(
    'jrsrp_20170430.csv',
    usecols=[
        'obs_key',
        "longitude",
        "latitude",
        "crust",
        "dist",
        "rock",
        "green",
        "crypto",
        "dead",
        "litter",
        "mid_g",
        "mid_d",
        "mid_b",
        "crn",
        "over_g",
        "over_d",
        "over_b",
        "persist",
        "num_points",
        "unoccluded",
        "image"
    ])

In [3]:
regression_coefficients = {
    'blue': [0.00041,0.97470],
    'green': [0.00289,0.99779],
    'red': [0.00274,1.00446],
    'nir': [0.00004,0.98906],
    'swir1': [0.00256,0.99467],
    'swir2': [-0.00327,1.02551]
}

In [4]:
measurements = [
    {'name': 'BS',
     'src_var': 'BS',
     'units': "percent",
     'dtype': 'int8',
     'nodata': -1,
     'zlib': True,
     'fletcher32': True,
     'attrs':{
         'long_name': "Bare ground fraction percentage",
         'comment': "Fraction of bare ground including rock, bare and disturbed soil, within the Landsat pixel as percentages",
         'alias': "bare",
         'coverage_content_type': 'modelResult'
     },
      'aliases':["bare"]},
    {'name': 'PV',
      'src_var': 'PV',
      'units': "percent",
      'dtype': 'int8',
      'nodata': -1,
      'zlib': True,
      'fletcher32': True,
      'attrs':{
          'long_name': "Green cover fraction percentage",
          'comment': "Fraction of green cover including green groundcover and green leaf material over all strata, within the Landsat pixel, as percentages",
          'alias': "green",
          'coverage_content_type': 'modelResult'
          },
      'aliases': ["green_veg"]},
    {'name': 'NPV',
      'src_var': 'NPV',
      'units': "percent",
      'dtype': 'int8',
      'nodata': -1,
      'zlib': True,
      'fletcher32': True,
      'attrs':{
          'long_name': "Non-green cover fraction percentage",
          'comment': "Fraction of non green cover including litter, dead leaf and branches over all strata, within the Landsat pixel, as percentages",
          'alias': "dead",
          'coverage_content_type': 'modelResult'
         },
      'aliases': ["dead_veg"]},
    {'name': 'UE',
      'src_var': 'UE',
      'units': "1",
      'dtype': 'int8',
      'nodata': -1,
      'zlib': True,
      'fletcher32': True,
      'attrs':{
          'long_name': "Unmixing error",
          'comment': "Eclidean Norm of the Residual Vector",
          'alias': "err",
          'coverage_content_type': 'qualityInformation'
          },
      'aliases': ["err"]}
]

In [5]:
def query_cube(obs_key, lon, lat, image):
    sensor = 'ls'+image[1:2]
    path = image[8:11]
    row = image[12:15]
    date = image[16:20]+'-'+image[20:22]+'-'+image[22:24]
    target_day = datetime.strptime(date, '%Y-%m-%d')
    day_before = (target_day - timedelta(days=1)).strftime('%Y-%m-%d')
    day_after = (target_day + timedelta(days=1)).strftime('%Y-%m-%d')
    
    query = {'time': (day_before, day_after)}
    
    wgs_84 = pyproj.Proj(init='epsg:4326')
    aus_albers = pyproj.Proj(init='epsg:3577')

    x, y = pyproj.transform(wgs_84, aus_albers, lon, lat)
    
    y_max = y-30
    y_min = y+30
    x_max = x-30
    x_min = x+30
    query['x'] = (x_min, x_max)
    query['y'] = (y_max, y_min)
    query['crs'] = 'EPSG:3577'
        
    query['source_filter'] = dict(product=sensor+'_nbart_scene', sat_path=path, sat_row=row)

    query['measurements'] = ['blue',
                           'green',
                           'red', 
                           'nir',
                           'swir1', 
                           'swir2'
                           ]
    
    data = dc.load(product=sensor+'_nbart_albers', **query)

    if(data.coords):
        data = data.squeeze()
        point = data.sel(x=x,y=y, method='nearest')
        new_x = point.coords['x'].values
        new_y = point.coords['y'].values
        
        new_y_max = new_y-30
        new_y_min = new_y+30
        new_x_max = new_x+30
        new_x_min = new_x-30
        query_x = (new_x_min, new_x_max)
        query_y = (new_y_min,new_y_max)
        target_cells = data.sel(x=slice(*query_x),y=slice(*query_y))
        
        min_lon, min_lat = pyproj.transform(aus_albers, wgs_84, new_x_min, new_y_min)
        max_lon, max_lat = pyproj.transform(aus_albers, wgs_84, new_x_max, new_y_max)
        
        if(sensor == 'ls8'):
             use_coefficients=regression_coefficients
        else:
             use_coefficients=None
        print('calculating new FC')
        target_cells = target_cells.squeeze()
        new_fc_data = fractional_cover(nbar_tile=target_cells, measurements=measurements, regression_coefficients=use_coefficients)
        
        BS_mean = float(new_fc_data.BS.mean())
        BS_std = float(new_fc_data.BS.std())
        PV_mean = float(new_fc_data.PV.mean())
        PV_std = float(new_fc_data.PV.std())
        NPV_mean = float(new_fc_data.NPV.mean())
        NPV_std = float(new_fc_data.NPV.std())
        UE_mean = float(new_fc_data.UE.mean())
        UE_std = float(new_fc_data.UE.std())
        
        x_size = new_fc_data.coords['x'].to_index().size
        y_size = new_fc_data.coords['y'].to_index().size
        
        return BS_mean, BS_std, PV_mean, PV_std, NPV_mean, NPV_std, UE_mean, UE_std, x_size * y_size
    else:
        return 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 0

In [7]:
with open('nbart_fieldsites_with_new_endmembers.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    columns = ['obs_key',
        'longitude',
        'latitude',
        'image',
        "crust",
        "dist",
        "rock",
        "green",
        "crypto",
        "dead",
        "litter",
        "mid_g",
        "mid_d",
        "mid_b",
        "crn",
        "over_g",
        "over_d",
        "over_b",
        "persist",
        "num_points",
        "unoccluded",
        "BS_mean",
        "BS_std",
        "PV_mean",
        "PV_std",
        "NPV_mean",
        "NPV_std",
        "UE_mean",
        "UE_std",
        'count'
        ]
    writer.writerow(columns)
    fieldsite_tuples = [tuple(x) for x in fieldsites.values]
    for fieldsite in fieldsite_tuples: 
        partialrow=(fieldsite[0],
                    fieldsite[1],
                    fieldsite[2],
                    fieldsite[3],
                    fieldsite[4],
                    fieldsite[5],
                    fieldsite[6],
                    fieldsite[7],
                    fieldsite[8],
                    fieldsite[9],
                    fieldsite[10],
                    fieldsite[11],
                    fieldsite[12],
                    fieldsite[13],
                    fieldsite[14],
                    fieldsite[15],
                    fieldsite[16],
                    fieldsite[17],
                    fieldsite[18],
                    fieldsite[19],
                    fieldsite[20])
        writer.writerow(partialrow+ query_cube(fieldsite[0],
                    fieldsite[1],
                    fieldsite[2],
                    fieldsite[20]))
        

calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating 

calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating 

calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating 

calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating 

calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating 

calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating 

calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating new FC
calculating 